In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None) # suppress unnecessary warnings
import numpy as np
from os import listdir
import re

import sqlalchemy as sa
import cx_Oracle

from pandas.tseries.offsets import Day, MonthEnd
from dateutil.relativedelta import relativedelta
import datetime
import os

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from chart_studio.plotly import plot, iplot
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

In [2]:
# Local directories
import_dir = 'D:/kay_data/data_marts/'
import_dir_parquet = 'D:/kay_data/data_marts/'
export_dir = import_dir
#export_dir = "C:/Users/khinmoetmoetnyein/Yoma Bank Limited/YB Digital - Documents/04 Working Folders/Design and Customer Experience/05_Design Projects/33_ATM Locations/"

def load_parquet(file_name):
    return pd.read_parquet(import_dir_parquet + file_name + '.parquet',engine='fastparquet')

def export_csv(export_df, export_file_name):
    return export_df.to_csv(os.path.join(export_dir + export_file_name + '.csv'))

def export_excel(export_df, export_file_name):
    return export_df.to_excel(os.path.join(export_dir + export_file_name + '.xlsx'))

def export_parquet(export_df, export_file_name):
    return export_df.to_parquet(os.path.join(export_dir + export_file_name + '.parquet'),allow_truncated_timestamps=True)

convert_timestamp_to_yyymmdd_string = lambda x: x.strftime("%Y-%m-%d") if pd.notnull(x) else None
reporting_date = str(datetime.datetime.now() + relativedelta(days=-2))[:10]

now = datetime.datetime.now()
version = "v" + str(now.year).zfill(2) + str(now.month).zfill(2) + str(now.day).zfill(2) +'_'

# Main Datasets

## Product

In [3]:
product_dwh = load_parquet('product_dwh')

In [4]:
fbe_product = load_parquet('fbe.product')#.rename(columns={'partyid':'cif'})

## Account

In [5]:
fbe_account = load_parquet('fbe.account').rename(columns={'customercode':'cif','branchcode':'branch_id',
                                                          'productcontextcode':'product_id'})
fbe_account = fbe_account.loc[fbe_account['opendate']<='2021-01-31']
fbe_account = fbe_account[~fbe_account['product_id'].str.contains('Internal')]#.product_id.unique()#.dropna(subset=['segmentation'])

In [6]:
fbe_account.segmentation.unique()

array(['LOCAL', 'SME', 'RETAIL', None, 'MNC'], dtype=object)

In [7]:
fbe_account.head(1)

,accountid,cif,branch_id,product_id,accountname,segmentation,isocurrencycode,status,creditlimit,debitlimit,...,closuredate,opendate,openedbyuser,reclastmodifieddate,reclastmodifiedby,accountisic,creditinterestrate,creditinterestmargin,debitinterestrate,debitinterestmargin
5228,002123070000002,02100034,00000021,42151LOCALMMK,"Myanmar V - Pile Co.,Ltd.",LOCAL,MMK,Activated,0.0,0.0,...,1970-01-01,2017-09-20,Y004784,2017-09-21 10:00:58.243,bftiuser,None,0.0,0.0,0.0,0.0


## Trade Finance Customer List

In [8]:
tf_pdt_list = product_dwh.loc[product_dwh['product_level2']=='Trade Finance'].product_id.unique()
tf_cust_list = fbe_account.loc[fbe_account['product_id'].isin(tf_pdt_list)].cif.unique()
del tf_pdt_list

## Customer

In [9]:
fbe_customer = load_parquet('fbe.customer').rename(columns={'partyid':'cif'})
fbe_customer = fbe_customer.loc[fbe_customer['createddatetime']<='2021-01-31']

In [10]:
party_sub_type = pd.read_excel(import_dir+'REF_PARTY_SUB_TYPE.xlsx')
party_sub_type.columns = ['partysubtype','party_sub_typ_desc']
party_sub_type.head(2)

,partysubtype,party_sub_typ_desc
0,ASSSOC,Associations and Societies incl NGO
1,COPSOC,Cooperative Societies


In [11]:
fbe_customer = fbe_customer.merge(party_sub_type,on='partysubtype',how='left')
fbe_customer.cif.count()

1017087

## Enterprise (for principal directors)

In [12]:
fbe_enterprise = load_parquet('fbe.enterprise').rename(columns={'partyid':'cif'})
fbe_enterprise.head(2)

,cif,mainbusinessactivity,dateformed,datetradingcommenced,businessstatus,registrationnumber,registrationcountry,noofbeneficialowners,dateofincorporation,noofprincipaldirectors,...,lastauditedon,isarchive,isdelete,nonresilicorpermitno,nonresilicorpermitnoissuedt,nonresilicorpermitnoexpirydt,lastapprovedby,lastmodifiedby,lastupdatedon,countryoftaxdomicile
0,10199449,EDUCATION_C,2018-11-13,2018-11-13,None,120500000,MMR,1,2018-11-13,2,...,NaT,N,N,None,NaT,NaT,Y002347,Y005009,2020-11-14 15:57:18.320,None
1,10199322,FISHINGAQUA,2016-09-27,2016-09-27,None,128634/2016-2017,MMR,1,2016-09-27,0,...,NaT,N,N,None,NaT,NaT,Y005406,Y006984,2020-11-26 18:53:26.737,None


## Loan disbursement

In [13]:
fbe_loan = load_parquet('fbe.loandisbursement')
fbe_loan = fbe_loan.loc[fbe_loan['disbursementdt']<='2021-01-31']
fbe_loan = fbe_loan.loc[fbe_loan['disbursedstatus']=='Y']
fbe_loan = fbe_loan[['disbursementid','disbursementamount']].rename(columns={'disbursementid':'accountid'})
fbe_loan = fbe_loan.merge(fbe_account[['cif','accountid']],on='accountid',how='left')
fbe_loan.head(2)

,accountid,disbursementamount,cif
0,005306980000203,910000.0,05301727
1,005306919000204,1358000.0,05301754


## Enterprise (for principal directors)

In [14]:
fbe_enterprise = load_parquet('fbe.enterprise').rename(columns={'partyid':'cif'})
fbe_enterprise.head(2)

,cif,mainbusinessactivity,dateformed,datetradingcommenced,businessstatus,registrationnumber,registrationcountry,noofbeneficialowners,dateofincorporation,noofprincipaldirectors,...,lastauditedon,isarchive,isdelete,nonresilicorpermitno,nonresilicorpermitnoissuedt,nonresilicorpermitnoexpirydt,lastapprovedby,lastmodifiedby,lastupdatedon,countryoftaxdomicile
0,10199449,EDUCATION_C,2018-11-13,2018-11-13,None,120500000,MMR,1,2018-11-13,2,...,NaT,N,N,None,NaT,NaT,Y002347,Y005009,2020-11-14 15:57:18.320,None
1,10199322,FISHINGAQUA,2016-09-27,2016-09-27,None,128634/2016-2017,MMR,1,2016-09-27,0,...,NaT,N,N,None,NaT,NaT,Y005406,Y006984,2020-11-26 18:53:26.737,None


In [15]:
fbe_account.loc[fbe_account['accountid']=='005310290500286']

,accountid,cif,branch_id,product_id,accountname,segmentation,isocurrencycode,status,creditlimit,debitlimit,...,closuredate,opendate,openedbyuser,reclastmodifieddate,reclastmodifiedby,accountisic,creditinterestrate,creditinterestmargin,debitinterestrate,debitinterestmargin
1430983,005310290500286,05301727,00000053,21001RETAILMMK,Than Zaw Oo,RETAIL,MMK,Activated,0.0,0.0,...,1970-01-01,2016-05-16,miguser,2020-07-26 21:23:52.603,EODUser1,None,0.0,0.0,13.0,0.0


In [16]:
fbe_product.loc[fbe_product['productid']=='21001RETAILMMK']

,subproductid,productid,productdescription,productdefaultdescription,productcategory,currency,bsheet_drnominalcode,bsheet_crnominalcode,pandl_nominalcode,gltype,...,has_stm,has_tax,has_trs,drparentnominalcode,drparentnominaldesc,crparentnominalcode,crparentnominaldesc,plparentnominalcode,plparentnominaldesc,threedigitcode
1285,21001,21001RETAILMMK,21001: Current Account-Customer,Current Account-Customer,CurrentAccount,MMK,A060010010,B030020010,None,SWING,...,Y,N,N,A060010,Standard Assets,B030020,Current Account,None,None,102


## Account Mandate (for signatories)

In [17]:
acc_mandate = load_parquet('ubtb_acctmandate').rename(columns={'ubcustomercode':'cif','ubaccountid':'accountid'})
acc_mandate.head(2)

,ubacctmandateidpk,accountid,cif,ubfinanciallimit,ublimitcurrencycode,ubmistxncode,ubnote,ubrole,ubisactive,versionnum
0,005344105500097P,005344105500097,05300116,0.0,MMK,<NA>,<NA>,OWNER,Y,0
1,005345143500041P,005345143500041,05300117,0.0,MMK,<NA>,<NA>,OWNER,Y,0


In [18]:
acc_mandate_pivot = acc_mandate.groupby('accountid').agg({'cif':'nunique'}).reset_index()
acc_mandate_pivot = acc_mandate_pivot.rename(columns={'cif':'no_of_signatories'})
acc_mandate_pivot = acc_mandate_pivot.merge(fbe_account[['accountid','cif']].drop_duplicates(),on='accountid',how='left')
acc_mandate_pivot.head(2)

,accountid,no_of_signatories,cif
0,000105210000005,1,10089637
1,000105210000013,1,10202659


## IDA

In [19]:
ida_dd = pd.read_excel(import_dir+'IDA Customer List - santitized_new.xlsx',
                       sheet_name='IDA_DD',dtype={'CIF':str}).rename(columns={'CIF':'cif'})
ida_mcct = pd.read_excel(import_dir+'IDA Customer List - santitized_new.xlsx',
                         sheet_name='IDA_MCCT',dtype={'CIF':str}).rename(columns={'CIF':'cif'})
ida = pd.concat([ida_dd[['cif']],ida_mcct[['cif']]]).drop_duplicates()
ida['ow'] = 'ida'
del ida_dd
del ida_mcct
print('Customers under IDA program - {} customers'.format(ida.cif.nunique()))

Customers under IDA program - 18828 customers


## Wave Linkage

In [20]:
wave_link1 = pd.read_csv(import_dir+'Wave-YOMA-Linkage-all-25-06-1969-28-09-2021.csv',dtype={'Bank Account Number(YOMA)':str})
wave_link2 = pd.read_csv(import_dir+'Wave-YOMA-Linkage-25-06-1969-28-09-2021.csv',dtype={'Bank Account Number(YOMA)':str})

wave_link = pd.concat([wave_link1,wave_link2])
wave_link = wave_link[['Bank Account Number(YOMA)',  'Linked Bank Status','UpdatedDate']].drop_duplicates()
wave_link.columns = ['accountid','linked_status','updated_date']
del wave_link1
del wave_link2
wave_link = wave_link.merge(fbe_account[['accountid','cif']],on='accountid',how='left')
wave_link = wave_link.loc[wave_link['linked_status']=='LINKED']

In [21]:
wave_link.linked_status.value_counts()

LINKED    154647
Name: linked_status, dtype: int64

In [22]:
print('Customers with SMART account linked to wave account - {} customers'.format(wave_link.cif.nunique()))

Customers with SMART account linked to wave account - 152581 customers


## DC active

In [23]:
dc_activated = load_parquet('fbe.dcactivated').rename(columns={'partyid':'cif'})

## SC & SCB

In [24]:
sc = pd.read_excel(import_dir+'SC All Disbursed Loans.xlsx',dtype={'CIF':str}).rename(columns={'CIF':'cif'})
sc = sc.loc[sc['Disbursement Date']<='2021-01-31']
print('Customers with at least 1 disbursed SC Personal loan - {} customers'.format(sc.cif.nunique()))

Customers with at least 1 disbursed SC Personal loan - 40938 customers


In [25]:
scb = pd.read_excel(import_dir+'SCB All Disbursed Loans.xlsx',dtype={'CIF':str}).rename(columns={'CIF':'cif'})
scb = sc.loc[sc['Disbursement Date']<='2021-01-31']
print('Customers with at least 1 disbursed SC Business loan - {} customers'.format(scb.cif.nunique()))

Customers with at least 1 disbursed SC Business loan - 40938 customers


## Transaction

In [26]:
credit_3_20lkh = load_parquet('credit_txn_between_3_20lkh')
print(credit_3_20lkh.month_end.min())
print(credit_3_20lkh.month_end.max())
print('Customers with 3-20lkh of credit transaction - {} customers'.format(credit_3_20lkh.cif.nunique()))

2020-02
2021-01
Customers with 3-20lkh of credit transaction - 200486 customers


In [27]:
pos_above_5 = load_parquet('pos_ecomm_yearly_txn_count_above_5')
pos_above_5 = pos_above_5[['cif']]
pos_above_5['ow'] = 'tun'
print('Customers with at least yearly 5 pos and ecomm transaction - {} customers'.format(pos_above_5.cif.nunique()))

Customers with at least yearly 5 pos and ecomm transaction - 6447 customers


In [28]:
topup_3k_less = load_parquet('mobile_topup_txn_less_than_30k')
print(topup_3k_less.month_end.min())
print(topup_3k_less.month_end.max())
print('Customers with less than 3k of mobile topup transaction - {} customers'
      .format(topup_3k_less.cif.nunique()))

2020-02
2021-01
Customers with less than 3k of mobile topup transaction - 131810 customers


In [29]:
topup_above_50_count = load_parquet('mobile_topup_txn_above_50')
print(topup_above_50_count.month_end.min())
print(topup_above_50_count.month_end.max())
print('Customers with above 50 mobile topup transaction count - {} customers'.format(topup_above_50_count.cif.nunique()))

2020-02
2021-01
Customers with above 50 mobile topup transaction count - 4668 customers


In [30]:
credit_20_50lkh = load_parquet('credit_txn_between_20_50lkh')
print(credit_20_50lkh.month_end.min())
print(credit_20_50lkh.month_end.max())
print('Customers with 20-50lkh of credit transaction - {} customers'.format(credit_20_50lkh.cif.nunique()))

2020-02
2021-01
Customers with 20-50lkh of credit transaction - 111933 customers


In [31]:
credit_3lkh_less = load_parquet('credit_txn_less_than_3lkh')
print(credit_3lkh_less.month_end.min())
print(credit_3lkh_less.month_end.max())
print('Customers with less than 3lkh of credit transaction - {} customers'.format(credit_3lkh_less.loc[credit_3lkh_less['credit_mmk']>1000].cif.nunique()))

2020-02
2021-01
Customers with less than 3lkh of credit transaction - 216998 customers


## Wave Agents

In [32]:
wave = load_parquet('wave_agent_dtr')
wave = wave.loc[wave['active']=='Y']
wave = wave.merge(fbe_account[['accountid','cif']],on='accountid',how='left')
wave = wave[['cif','type']].rename(columns={'type':'wave'})

In [33]:
wave.head()

,cif,wave
0,10227202,POS
1,10196056,POS
2,10302092,POS
3,10304902,POS
4,10235707,POS


## Weekly balance

In [34]:
#getting the list of accounts under deposit products
deposit_acc = fbe_account[fbe_account['product_id'].isin(product_dwh.loc[product_dwh['product_level1']=='Deposit'].product_id.unique())].accountid.unique()

path = r"D:/kay_data/data_marts"
files = os.listdir(path)
files_parquet = ['202002_bal.parquet','202003_bal.parquet','202004_bal.parquet','202005_bal.parquet','202006_bal.parquet',
                 '202007_bal.parquet','202008_bal.parquet','202009_bal.parquet','202010_bal.parquet','202011_bal.parquet',
                 '202012_bal.parquet','202101_bal.parquet']

#getting weekly date range
date_range = pd.date_range(start='2020-02-01',periods=52, freq='W')


bal = pd.DataFrame()
for f in files_parquet:
    temp_bal = pd.read_parquet(path + '/' +f)
    #getting the balance records on weekly basis on predefined date range above
    temp_bal = temp_bal.loc[temp_bal['balance_date'].isin(date_range)]
    #deposit accounts only
    temp_bal = temp_bal.loc[temp_bal['account_id'].isin(deposit_acc)]
    #making sure current overdraft accounts and its balances are excluded 
    temp_bal['closing_balance_mmk'] = np.where(temp_bal['closing_balance_mmk']<0,0,temp_bal['closing_balance_mmk'])
    #combining the files
    bal = pd.concat([bal,temp_bal])

In [35]:
#wave = load_parquet('wave_agent_dtr')

In [36]:
week_bal = bal.merge(fbe_account[['accountid','cif']],left_on='account_id',right_on='accountid',
                          how='left').drop(columns=['account_id'])
weekly_bal_cif = week_bal.groupby(['balance_date','cif']).closing_balance_mmk.sum().reset_index()
weekly_bal_cif = weekly_bal_cif.sort_values(by=['cif','balance_date'])
weekly_bal_cif['diff'] = weekly_bal_cif.groupby(['cif'])['closing_balance_mmk'].diff().fillna(0)
weekly_bal_cif['diff'] = np.where(weekly_bal_cif['diff'].abs() <= 10000,0,weekly_bal_cif['diff'])
del week_bal
print(weekly_bal_cif.balance_date.min())
print(weekly_bal_cif.balance_date.max())

2020-02-02 00:00:00
2021-01-24 00:00:00


In [37]:
pct_change = weekly_bal_cif.pivot(index='cif',columns='balance_date',values='closing_balance_mmk').fillna(0)
pct_change = pct_change.pct_change(axis='columns')
pct_change[np.isneginf(pct_change)] = 0
pct_change[np.isinf(pct_change)] = 0
pct_change['avg'] = pct_change.abs().mean(axis=1)
pct_change = np.round(pct_change,decimals=2)

In [38]:
mid_range_minthu = pct_change.loc[(pct_change['avg']>=0.2)&(pct_change['avg']<=0.5)].reset_index()[['cif']]
high_range_zaw = pct_change.loc[(pct_change['avg']>=0.5)&(pct_change['avg']<=0.9)].reset_index()[['cif']]
highest_range_aungnaing = pct_change.loc[(pct_change['avg']>0.9)].reset_index()[['cif']]
print('Middle range of week on week balance variance - {} customers'.format(mid_range_minthu.cif.nunique()))
print('High range of week on week balance variance - {} customers'.format(high_range_zaw.cif.nunique()))
print('Highest range of week on week balance variance - {} customers'.format(highest_range_aungnaing.cif.nunique()))

Middle range of week on week balance variance - 47592 customers
High range of week on week balance variance - 28307 customers
Highest range of week on week balance variance - 153088 customers


# Analysis

## Latt Latt

In [39]:
print('Customers who are wave agents - {} customers'.format(wave.loc[wave['wave']!='DTR'].cif.nunique()))
print('Customers with above 50 mobile topup transaction count - {} customers'.format(topup_above_50_count.cif.nunique()))

Customers who are wave agents - 23904 customers
Customers with above 50 mobile topup transaction count - 4668 customers


In [40]:
latt_latt = wave.loc[wave['wave']!='DTR'][['cif']]
latt_latt['ow'] = 'wave_agent'
latt_latt = latt_latt.append(topup_above_50_count[['cif']]).drop_duplicates()
latt_latt = latt_latt[~latt_latt['cif'].isin(tf_cust_list)]
latt_latt['persona'] = 'latt_latt'
latt_latt.head()

,cif,ow,persona
0,10227202,wave_agent,latt_latt
1,10196056,wave_agent,latt_latt
2,10302092,wave_agent,latt_latt
3,10304902,wave_agent,latt_latt
4,10235707,wave_agent,latt_latt


In [41]:
print('No of customers under persona; Latt Latt - {} customers'.format(latt_latt.cif.nunique()))

No of customers under persona; Latt Latt - 28466 customers


## Min Thu

In [42]:
hp_productid = fbe_product.loc[fbe_product['productcategory']=='HirePurchase'].productid.unique()
hp = fbe_account.loc[fbe_account['product_id'].isin(hp_productid)]
del hp_productid
print('Customers with at least 1 HP product - {} customers'.format(hp.cif.nunique()))

Customers with at least 1 HP product - 117939 customers


In [43]:
print('Middle range of week on week balance variance - {} customers'.format(mid_range_minthu.cif.nunique()))

Middle range of week on week balance variance - 47592 customers


In [44]:
branch_visit = load_parquet('branch_visit_above_3')
print(branch_visit.month_end.min())
print(branch_visit.month_end.max())

2020-02
2021-01


In [45]:
branch_visit_pivot = branch_visit.loc[branch_visit['cif'].isin(fbe_customer[['cif','customersegment']].dropna().cif.unique())]
branch_visit_pivot = branch_visit_pivot.pivot(index='cif',columns='month_end',values='no_of_branch_visit').reset_index()
branch_visit_pivot = branch_visit_pivot.dropna(thresh=2) #keeping rows that visited the branch at least 3 months in the past year
del branch_visit
print('Customers with at least 3 different brach visits in a month - {} customers'.format(branch_visit_pivot.cif.nunique()))

Customers with at least 3 different brach visits in a month - 5311 customers


In [46]:
min_thu = pd.concat([hp[['cif']],branch_visit_pivot[['cif']],mid_range_minthu[['cif']]]).drop_duplicates()
min_thu = min_thu[~min_thu['cif'].isin(tf_cust_list)]
min_thu['persona'] = 'min_thu'

In [47]:
print('No of customers under persona; Min Thu - {} customers'.format(min_thu.cif.nunique()))

No of customers under persona; Min Thu - 165400 customers


## Zaw

In [48]:
#print('Customers with above 50 mobile topup transaction count - {} customers'.format(c.cif.nunique()))

In [49]:
print('Customers who are wave agents - {} customers'.format(wave.loc[wave['wave']=='DTR'].cif.nunique()))

Customers who are wave agents - 277 customers


In [50]:
print('Customers with SCB loan - {} customers'.format(scb.cif.nunique()))

Customers with SCB loan - 40938 customers


In [51]:
sig_zaw = acc_mandate_pivot.loc[(acc_mandate_pivot['no_of_signatories']==2)][['cif']]
print('Customers with 2 authorized signatories - {} customers'.format(sig_zaw.cif.nunique()))

Customers with 2 authorized signatories - 76786 customers


In [52]:
zaw = wave.loc[wave['wave']=='DTR'][['cif']]
#zaw = zaw.append(sig_zaw[['cif']].drop_duplicates())
zaw['ow'] = 'dtr'
zaw = zaw.append(scb[['cif']].drop_duplicates()).append(sig_zaw[['cif']].drop_duplicates())
zaw = zaw.drop_duplicates(subset=['cif'],keep='first')
zaw['persona'] = 'zaw'
zaw.head()

,cif,ow,persona
23906,10584993,dtr,zaw
23907,10414052,dtr,zaw
23908,10387943,dtr,zaw
23909,10236556,dtr,zaw
23910,10056683,dtr,zaw


In [53]:
#sig = acc_mandate_pivot.loc[(acc_mandate_pivot['no_of_signatories']==2)]#.dropna(subset=['modeofoperation'])
#print('Customers with at least 2 authorized signatories - {} customers'.format(sig.cif.nunique()))

In [54]:
print('No of customers under persona; Zaw - {} customers'.format(zaw.cif.nunique()))

No of customers under persona; Zaw - 116441 customers


## Phyo

In [55]:
corp = fbe_customer.loc[fbe_customer['customersegment'].isin(['LOCAL','MNC'])]
corp = corp[['cif']]
corp['ow'] = 'corp'
print('Customers under Corporate customer segment - {} customers'.format(corp.cif.nunique()))

Customers under Corporate customer segment - 598 customers


In [56]:
fx = fbe_account.loc[(fbe_account['segmentation'].isin(['LOCAL', 'SME','MNC']))&(fbe_account['isocurrencycode']!='MMK')]
fx = fx[['cif']]
fx['ow'] = 'fx'
print('Customers with at least 1 enterprise fx account - {} customers'.format(fx.cif.nunique()))

Customers with at least 1 enterprise fx account - 1202 customers


In [57]:
loan_1b = fbe_loan.loc[fbe_loan['disbursementamount']>=1000000000][['cif']]
loan_1b['ow'] = 'loan_1b'
print('Customers with at least 1bil loan account - {} customers'.format(loan_1b.cif.nunique()))

Customers with at least 1bil loan account - 65 customers


In [58]:
sig = acc_mandate_pivot.loc[(acc_mandate_pivot['no_of_signatories']>1)][['cif']]
print('Customers with at least 2 authorized signatories - {} customers'.format(sig.cif.nunique()))

Customers with at least 2 authorized signatories - 92259 customers


In [59]:
director = fbe_enterprise.loc[fbe_enterprise['noofprincipaldirectors']>1][['cif']]
print('Customers with at least 2 principal directors - {} customers'.format(director.cif.nunique()))

Customers with at least 2 principal directors - 8510 customers


In [60]:
phyo = pd.concat([corp,fx,loan_1b]).drop_duplicates(subset=['cif'],keep='first')
phyo = phyo.append(pd.concat([sig,director]).drop_duplicates()).drop_duplicates()
phyo['persona'] = 'phyo'
phyo.head()

,cif,ow,persona
4325,00108316,corp,phyo
18059,02502042,corp,phyo
161236,01100347,corp,phyo
656306,04806607,corp,phyo
656314,01707595,corp,phyo


In [61]:
phyo.ow.value_counts(dropna=False)

NaN        93225
fx          1024
corp         598
loan_1b       12
Name: ow, dtype: int64

In [62]:
print('No of customers under persona; Phyo - {} customers'.format(phyo.cif.nunique()))

No of customers under persona; Phyo - 93243 customers


## Aung Naing

In [63]:
print('Customers with SC personal loan of 2 mil+ - {} customers'.format(sc.loc[sc['Disbursed Amount']>=2000000].cif.nunique()))

Customers with SC personal loan of 2 mil+ - 4756 customers


In [64]:
call_dep_prod_list = fbe_product.loc[fbe_product['productcategory']=='CallDeposit'].productid.unique()
call_dep = fbe_account.loc[fbe_account['product_id'].isin(call_dep_prod_list)][['cif']]
del call_dep_prod_list
print('Customers with Call deposit product - {} customers'.format(call_dep.cif.nunique()))

Customers with Call deposit product - 71113 customers


In [65]:
highest_range_aungnaing = pct_change.loc[(pct_change['avg']>0.9)].reset_index()[['cif']]
print('Customers with w-on-w deposit variation above 90% - {} customers'.format(highest_range_aungnaing.cif.nunique()))

Customers with w-on-w deposit variation above 90% - 153088 customers


In [66]:
aung_naing = pd.concat([sc.loc[sc['Disbursed Amount']>=2000000][['cif']],
                        highest_range_aungnaing[['cif']],call_dep[['cif']]]).drop_duplicates()
aung_naing['persona'] = 'aung_naing'

In [67]:
print('No of customers under persona; Aung Naing - {} customers'.format(aung_naing.cif.nunique()))

No of customers under persona; Aung Naing - 203690 customers


## Moe Thuzar

In [68]:
print('Customers under IDA program - {} customers'.format(ida.cif.nunique()))

Customers under IDA program - 18828 customers


In [69]:
print('Customers with less than 3lkh of credit transaction - {} customers'.format(credit_3lkh_less.loc[credit_3lkh_less['credit_mmk']>1000].cif.nunique()))

Customers with less than 3lkh of credit transaction - 216998 customers


In [70]:
moe_thuzar = ida.copy()
moe_thuzar = moe_thuzar.append(credit_3lkh_less.loc[credit_3lkh_less['credit_mmk']>1000][['cif']].drop_duplicates())
moe_thuzar = moe_thuzar[~moe_thuzar['cif'].isin(tf_cust_list)]
moe_thuzar['persona'] = 'moe_thuzar'
moe_thuzar.head(2)

,cif,ow,persona
0,10074560,ida,moe_thuzar
1,10102508,ida,moe_thuzar


In [71]:
print('No of customers under persona; Moe Thuzar - {} customers'.format(moe_thuzar.cif.nunique()))

No of customers under persona; Moe Thuzar - 231192 customers


## Htwe

In [72]:
personal = fbe_customer.loc[fbe_customer['customersegment'].isin(['RETAIL'])]
print('Customers under PERSONAL segment - {} customers'.format(personal.cif.nunique()))

Customers under PERSONAL segment - 1002668 customers


In [73]:
sav_fd_list = fbe_product.loc[fbe_product['productcategory'].isin(['savings','fixdep'])].productid.unique()
sav_fd = fbe_account.loc[fbe_account['product_id'].isin(sav_fd_list)]
sav_fd = sav_fd.loc[sav_fd['cif'].isin(personal.cif.unique())]
del sav_fd_list
del personal
print('Customers with Savings or Fixed Deposit account - {} customers'.format(sav_fd.cif.nunique()))

Customers with Savings or Fixed Deposit account - 432414 customers


In [74]:
credit_20_50lkh = load_parquet('credit_txn_between_20_50lkh')
print(credit_20_50lkh.month_end.min())
print(credit_20_50lkh.month_end.max())
print('Customers with 20-50lkh of credit transaction - {} customers'.format(credit_20_50lkh.cif.nunique()))

2020-02
2021-01
Customers with 20-50lkh of credit transaction - 111933 customers


In [75]:
htwe = sav_fd[['cif']]
htwe['ow'] = np.nan
htwe = htwe.append(credit_20_50lkh[['cif']].drop_duplicates())
htwe = htwe[~htwe['cif'].isin(tf_cust_list)]
htwe['persona'] = 'htwe'
htwe.head(2)

,cif,ow,persona
9002,05300156,NaN,htwe
9003,05301013,NaN,htwe


In [76]:
print('No of customers under persona; Htwe - {} customers'.format(htwe.cif.nunique()))

No of customers under persona; Htwe - 480940 customers


## Tun

In [77]:
onboarding = fbe_customer.loc[fbe_customer['relationshipmgrid'].isin(['RemoteSignUp','PayrollSignUp'])]
onboarding = onboarding[['cif','relationshipmgrid']].rename(columns={'relationshipmgrid':'ow'})
onboarding['ow'] = onboarding['ow'].replace({'PayrollSignUp':np.nan,'RemoteSignUp':'ApplySMART'})
onboarding['ow'].value_counts(dropna=False)

ApplySMART    22544
NaN           10200
Name: ow, dtype: int64

In [78]:
print('Customers with SC personal loan of below 2 mil - {} customers'.format(sc.loc[sc['Disbursed Amount']<2000000].cif.nunique()))

Customers with SC personal loan of below 2 mil - 39749 customers


In [79]:
print('Customers with SMART account linked to wave account - {} customers'.format(wave_link.cif.nunique()))

Customers with SMART account linked to wave account - 152581 customers


In [80]:
onboarding.head()

,cif,ow
295677,10148641,ApplySMART
295691,10149105,ApplySMART
295692,10149151,ApplySMART
295693,10149152,ApplySMART
295713,10138567,ApplySMART


In [81]:
tun = onboarding.copy()
tun = tun.append(sc.loc[sc['Disbursed Amount']<2000000][['cif']].drop_duplicates())
tun = tun.append(wave_link[['cif']].drop_duplicates())
tun = tun[~tun['cif'].isin(tf_cust_list)]
tun['persona'] = 'tun'
tun.head(2)

,cif,ow,persona
295677,10148641,ApplySMART,tun
295691,10149105,ApplySMART,tun


In [82]:
print('No of customers under persona; Tun - {} customers'.format(tun.cif.nunique()))

No of customers under persona; Tun - 189622 customers


# Anti-Persona

In [83]:
anti = fbe_customer.loc[fbe_customer['party_sub_typ_desc'].isin(['Associations and Societies incl NGO','Embassy'])]
anti = anti[['cif']]
anti['ow'] = 'anti_persona'
anti['persona'] = 'anti_persona'
anti['ow_count'] = 2

In [84]:
print('No of customers under anti-persona - {} customers'.format(anti.cif.nunique()))

No of customers under anti-persona - 184 customers


# Persona

In [85]:
persona = pd.concat([latt_latt,min_thu,zaw,phyo,aung_naing,moe_thuzar,htwe,tun])
persona['ow_count'] = np.where(persona['ow'].isnull(),0,1)
persona = persona.sort_values(by=['cif','ow_count']).drop_duplicates(subset=['cif'],keep='last')
persona = persona.append(anti).drop_duplicates(subset=['cif'],keep='last')
print('No of customers; bank-wide total - {} customers'.format(fbe_customer.cif.nunique()))
print('No of customers covered by persona + anti-persona - {} customers'.format(persona.cif.nunique()))
print('% covered by persona + anti-persona - {r:1.0f}%'.format(r=(persona.cif.nunique()*100)/(fbe_customer.cif.nunique())))
print('No of customers in outliers group - {} customers'.format(fbe_customer.cif.nunique()-persona.cif.nunique()))

No of customers; bank-wide total - 1017087 customers
No of customers covered by persona + anti-persona - 758941 customers
% covered by persona + anti-persona - 75%
No of customers in outliers group - 258146 customers


In [86]:
persona.groupby('persona').agg({'cif':'nunique'}).rename(columns={'cif':'unique_cust_count'})

,unique_cust_count
persona,
anti_persona,184
aung_naing,36067
htwe,363721
latt_latt,22562
min_thu,93092
moe_thuzar,66965
phyo,9660
tun,166418
zaw,272


In [87]:
9531-9650

-119

In [88]:
persona.loc[persona['cif']=='10274222']

,cif,ow,persona,ow_count
941151,10274222,ApplySMART,tun,1


In [89]:
persona.cif.value_counts()

10360700    1
10173558    1
10145164    1
10503389    1
10645648    1
           ..
10526674    1
10604616    1
03005502    1
10517340    1
10366635    1
Name: cif, Length: 758941, dtype: int64

# Derived fields

In [90]:
total_txn_incl = load_parquet('total_txn_count_incl_system_txn')
total_txn_incl.head()

,cif,month_end,total_transaction_count
0,10292427,2020-02,2
1,10080386,2020-02,2
2,02604087,2020-02,2
3,10095235,2020-02,18
4,01606978,2020-02,2


In [91]:
total_txn_incl = load_parquet('total_txn_count_incl_system_txn')

mean_txn_incl = total_txn_incl.groupby('cif').agg({'total_transaction_count':'mean'}).rename(columns={'total_transaction_count':'avg_txn_count_incl_mean'})
median_txn_incl = pd.DataFrame(total_txn_incl.groupby('cif')[['total_transaction_count']].apply(np.median)).rename(columns={0:'avg_txn_count_incl_median'})
avg_txn_incl = mean_txn_incl.join(median_txn_incl)
del mean_txn_incl
del median_txn_incl
del total_txn_incl
avg_txn_incl.head()

,avg_txn_count_incl_mean,avg_txn_count_incl_median
cif,,
00000001,68039.416667,69431.0
00000002,65403.833333,64088.5
00000003,48368.166667,47642.5
00000004,58571.916667,56686.5
00000005,30586.250000,29613.0


In [92]:
total_txn_excl = load_parquet('total_txn_count_excl_system_txn')

mean_txn_excl = total_txn_excl.groupby('cif').agg({'total_transaction_count':'mean'}).rename(columns={'total_transaction_count':'avg_txn_count_excl_mean'})
median_txn_excl = pd.DataFrame(total_txn_excl.groupby('cif')[['total_transaction_count']].apply(np.median)).rename(columns={0:'avg_txn_count_excl_median'})
avg_txn_excl = mean_txn_excl.join(median_txn_excl)
del mean_txn_excl
del median_txn_excl
del total_txn_excl
avg_txn_excl.head()

,avg_txn_count_excl_mean,avg_txn_count_excl_median
cif,,
00000001,6847.666667,7473.0
00000002,12224.916667,13040.5
00000003,5875.000000,6461.0
00000004,7692.500000,7861.5
00000005,5501.666667,4715.5


In [93]:
total_deposit = load_parquet('total_monthly_deposit_txn_amount')

mean_deposit = total_deposit.groupby('cif').agg({'deposit_txn_amount':'mean'}).rename(columns={'deposit_txn_amount':'avg_deposit_txn_amount_mean'})
median_deposit = pd.DataFrame(total_deposit.groupby('cif')[['deposit_txn_amount']].apply(np.median)).rename(columns={0:'avg_deposit_txn_amount_median'})
avg_deposit = np.round(mean_deposit.join(median_deposit)/1e6,decimals=2)
avg_deposit.columns = avg_deposit.columns + '_in_mil'
del mean_deposit
del median_deposit
del total_deposit
avg_deposit.head()

,avg_deposit_txn_amount_mean_in_mil,avg_deposit_txn_amount_median_in_mil
cif,,
00000001,30824.33,27955.49
00000002,839.11,683.23
00000003,623.81,293.87
00000004,145.08,153.46
00000005,4031.77,4004.91


In [94]:
total_withdrawal = load_parquet('total_monthly_withdrawal_txn_amount')

mean_withdrawal = total_withdrawal.groupby('cif').agg({'withdrawl_txn_amount':'mean'}).rename(columns={'withdrawl_txn_amount':'avg_withdrawal_txn_amount_mean'})
median_withdrawal = pd.DataFrame(total_withdrawal.groupby('cif')[['withdrawl_txn_amount']].apply(np.median)).rename(columns={0:'avg_withdrawal_txn_amount_median'})
avg_withdrawal = np.round((mean_withdrawal.join(median_withdrawal))/1e6,decimals=2)
avg_withdrawal.columns = avg_withdrawal.columns + '_in_mil'
del mean_withdrawal
del median_withdrawal
del total_withdrawal
avg_withdrawal.head()

,avg_withdrawal_txn_amount_mean_in_mil,avg_withdrawal_txn_amount_median_in_mil
cif,,
00000001,36168.42,35021.27
00000002,489.37,380.39
00000003,3390.11,3192.44
00000004,250.08,236.90
00000005,345.40,288.27


In [95]:
acc_pdt = fbe_account.loc[fbe_account['status']=='Activated']
acc_pdt = acc_pdt[['cif','accountid','product_id']].merge(product_dwh[['product_id','product_level1','product_level2']],on='product_id',how='left').dropna()
level1 = acc_pdt.loc[acc_pdt['product_level1'].isin(['Deposit','Loan'])].pivot_table(index='cif',columns='product_level1',values='accountid',aggfunc='nunique')
level1.columns = ['no_of_deposit_products','no_of_loan_products'] 

In [96]:
level2 = acc_pdt.loc[acc_pdt['product_level1'].isin(['Deposit','Loan'])].pivot_table(index='cif',columns='product_level2',values='accountid',aggfunc='nunique')

In [97]:
product_dwh.loc[(product_dwh['product_level1']!='Deposit')&(product_dwh['product_level1']!='Loan')][['product_name','product_level1','product_level2']]

,product_name,product_level1,product_level2
0,60002: Interest on Fixed Deposit - Accrual,None,None
1,60003: Interest on Call Deposit - Accrual,None,None
2,60011: Interest Expense on J Zu Account - Accrual,None,None
3,60004: Interest on Savings Deposit - Application,None,None
4,60005: Interest on Fixed Deposit - Application,None,None
...,...,...,...
5030,Interest in Suspense on - Trade Finance,None,None
5031,42104:Unrealized Gains/Losses-HTM-Contra-THB,None,None
5033,Demand Loan-Contract Financing(Corporate-Secured),None,None
5035,Demand Loan-Contract Financing(SME Secured),None,None


In [98]:
tagging = fbe_customer[['cif']].drop_duplicates()
tagging = tagging.merge(persona[['cif','persona']].drop_duplicates(),on='cif',how='left')
tagging['persona'] = tagging['persona'].fillna('outliers')
tagging = tagging.set_index('cif').join(avg_deposit).join(avg_withdrawal).join(avg_txn_excl).join(avg_txn_incl).join(level2)
export_csv(tagging,version+'persona_with_derived_fields')
#tagging = tagging.drop(columns=['avg_deposit_txn_amount_median_in_mil','avg_withdrawal_txn_amount_median_in_mil','avg_txn_count_excl_median', 'avg_txn_count_incl_median'])
tagging = tagging.reset_index()
tagging.head(2)

,cif,persona,avg_deposit_txn_amount_mean_in_mil,avg_deposit_txn_amount_median_in_mil,avg_withdrawal_txn_amount_mean_in_mil,avg_withdrawal_txn_amount_median_in_mil,avg_txn_count_excl_mean,avg_txn_count_excl_median,avg_txn_count_incl_mean,avg_txn_count_incl_median,...,Demand Loan,Fixed Deposit,Foreign Currency Deposit,Hire Purchase,J'Zu,Overdraft,Savings Deposit,Smart,Staff Loan,Trade Finance
0,05300695,htwe,NaN,NaN,NaN,NaN,NaN,NaN,1.750000,1.5,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN
1,05300696,tun,0.06,0.06,0.63,0.37,26.0,28.0,32.583333,34.5,...,NaN,NaN,NaN,1.0,1.0,NaN,1.0,1.0,1.0,NaN


In [99]:
tagging.loc[tagging['persona'].isin(['latt_latt','min_thu','moe_thuzar','htwe','tun'])][['Trade Finance']].sum()

Trade Finance    0.0
dtype: float64

In [100]:
outliers = tagging.loc[tagging['persona']=='outliers']
outliers_txn = outliers[['cif','avg_deposit_txn_amount_mean_in_mil','avg_withdrawal_txn_amount_mean_in_mil','avg_txn_count_excl_mean','avg_txn_count_incl_mean']]
outliers_product = outliers[['cif','Call Deposit', 'Current', 'Demand Loan', 'Fixed Deposit', 'Foreign Currency Deposit', 'Hire Purchase', "J'Zu", 'Overdraft', 'Savings Deposit', 'Smart', 'Staff Loan', 'Trade Finance']]
outliers_product.head()

,cif,Call Deposit,Current,Demand Loan,Fixed Deposit,Foreign Currency Deposit,Hire Purchase,J'Zu,Overdraft,Savings Deposit,Smart,Staff Loan,Trade Finance
12,05300768,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,05300860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,05300861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,05300874,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,05300942,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [101]:
outliers_product.set_index('cif').dropna(how='all',axis='index').reset_index().cif.nunique()

85552

In [102]:
print("Total customers in outliers group - {} customers".format(outliers.cif.nunique()))
print("Total customers who did not do any transactions in outliers group - {} customers".format((outliers.dropna(subset=['avg_txn_count_incl_mean']).cif.nunique())))
print("Total customers with less than 10k (monthly) deposit or withdrawal in outliers group - {} customers".format(outliers.loc[(outliers['avg_withdrawal_txn_amount_mean_in_mil']<=(10000/1e6))|(outliers['avg_deposit_txn_amount_mean_in_mil']<=(10000/1e6))].dropna(subset=['avg_txn_count_incl_mean']).cif.nunique()))

Total customers in outliers group - 258266 customers
Total customers who did not do any transactions in outliers group - 71463 customers
Total customers with less than 10k (monthly) deposit or withdrawal in outliers group - 1903 customers


In [103]:
product_dwh.loc[product_dwh['product_level2']=='Demand Loan'][['product_level1','product_level2','product_level3']].drop_duplicates()

,product_level1,product_level2,product_level3
1269,Loan,Demand Loan,Home Loan / Mortgage
1284,Loan,Demand Loan,Demand Loan
1496,Loan,Demand Loan,MFI
1913,Loan,Demand Loan,Digital Credit


In [104]:
outliers.loc[(outliers['avg_withdrawal_txn_amount_mean_in_mil']<=0.01)].dropna(subset=['avg_txn_count_incl_mean']).avg_withdrawal_txn_amount_mean_in_mil.max()

0.01

In [105]:
71460/258262

0.27669575857075374

In [106]:
summary_mean = np.round((tagging[['persona','avg_deposit_txn_amount_mean_in_mil','avg_withdrawal_txn_amount_mean_in_mil','avg_txn_count_excl_mean','avg_txn_count_incl_mean']].groupby('persona').mean()),decimals=0)
summary_mean

,avg_deposit_txn_amount_mean_in_mil,avg_withdrawal_txn_amount_mean_in_mil,avg_txn_count_excl_mean,avg_txn_count_incl_mean
persona,,,,
anti_persona,25.0,24.0,4.0,5.0
aung_naing,34.0,27.0,8.0,7.0
htwe,11.0,8.0,5.0,4.0
latt_latt,21.0,17.0,109.0,284.0
min_thu,4.0,2.0,2.0,2.0
moe_thuzar,1.0,3.0,4.0,5.0
outliers,11.0,2.0,4.0,3.0
phyo,121.0,194.0,339.0,154.0
tun,4.0,3.0,20.0,19.0


In [107]:
np.round(tagging.loc[tagging['persona']=='htwe'].describe(),decimals=0).T.drop(columns=['count'])

,mean,std,min,25%,50%,75%,max
avg_deposit_txn_amount_mean_in_mil,11.0,70.0,0.0,0.0,1.0,5.0,10723.0
avg_deposit_txn_amount_median_in_mil,9.0,65.0,0.0,0.0,1.0,4.0,11136.0
avg_withdrawal_txn_amount_mean_in_mil,8.0,83.0,0.0,0.0,1.0,4.0,16157.0
avg_withdrawal_txn_amount_median_in_mil,7.0,97.0,0.0,0.0,1.0,3.0,19700.0
avg_txn_count_excl_mean,5.0,15.0,1.0,1.0,2.0,5.0,3587.0
avg_txn_count_excl_median,5.0,15.0,1.0,1.0,2.0,4.0,3587.0
avg_txn_count_incl_mean,4.0,16.0,1.0,2.0,2.0,3.0,4799.0
avg_txn_count_incl_median,4.0,12.0,1.0,2.0,2.0,3.0,3656.0
Call Deposit,1.0,0.0,1.0,1.0,1.0,1.0,6.0
Current,1.0,0.0,1.0,1.0,1.0,1.0,9.0


In [108]:
persona_list = tagging.persona.unique()
persona_list

array(['htwe', 'tun', 'outliers', 'aung_naing', 'min_thu', 'latt_latt',
       'phyo', 'moe_thuzar', 'zaw', 'anti_persona'], dtype=object)

In [109]:
df = pd.DataFrame()
for persona in persona_list:
    temp = np.round(tagging.loc[tagging['persona']==persona].describe(),decimals=0).T.drop(columns=['count'])
    temp['persona'] = persona
    df = pd.concat([df,temp])

In [110]:
df.head()

,mean,std,min,25%,50%,75%,max,persona
avg_deposit_txn_amount_mean_in_mil,11.0,70.0,0.0,0.0,1.0,5.0,10723.0,htwe
avg_deposit_txn_amount_median_in_mil,9.0,65.0,0.0,0.0,1.0,4.0,11136.0,htwe
avg_withdrawal_txn_amount_mean_in_mil,8.0,83.0,0.0,0.0,1.0,4.0,16157.0,htwe
avg_withdrawal_txn_amount_median_in_mil,7.0,97.0,0.0,0.0,1.0,3.0,19700.0,htwe
avg_txn_count_excl_mean,5.0,15.0,1.0,1.0,2.0,5.0,3587.0,htwe


In [111]:
export_csv(df,version+'mean_median_by_persona')

In [112]:
level2['total'] = level2.sum(axis='columns')

In [113]:
level2.loc[level2['total']==0]

product_level2,Call Deposit,Current,Demand Loan,Fixed Deposit,Foreign Currency Deposit,Hire Purchase,J'Zu,Overdraft,Savings Deposit,Smart,Staff Loan,Trade Finance,total
cif,,,,,,,,,,,,,


In [114]:
level2.head()

product_level2,Call Deposit,Current,Demand Loan,Fixed Deposit,Foreign Currency Deposit,Hire Purchase,J'Zu,Overdraft,Savings Deposit,Smart,Staff Loan,Trade Finance,total
cif,,,,,,,,,,,,,
00100001,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
00100002,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
00100004,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
00100005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
00100007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0


# Appendix 

In [115]:
abc.shape()

NameError: name 'abc' is not defined

In [ ]:
median_withdrawal = pd.DataFrame(total_withdrawal.groupby('cif')[['withdrawl_txn_amount']].apply(np.median)).rename(columns={0:'avg_withdrawal_txn_amount_median'})


In [ ]:
fixdep = load_parquet('fbe.fixdep')

In [ ]:
x = acc_pdt.loc[acc_pdt['cif'].isin(zaw.cif.unique())]
y = x.loc[x['product_level1']=='Deposit'].pivot_table(index='cif',columns='product_level2',values='accountid',aggfunc='nunique')
y['total'] = y.sum(axis='columns')
y = y.sort_values(by='total',ascending=False)

In [ ]:
print(y.reset_index().cif.nunique())
print(y.loc[y['total']>50].reset_index().cif.nunique())

In [ ]:
y.head(20)

In [ ]:
a = load_parquet('202101_bal')

In [ ]:
fixdep.status.value_counts(dropna=False)

In [ ]:
zaw_acc = a.merge(fbe_account.loc[fbe_account['cif']=='10183500'][['accountid','cif','product_id']],how='inner',left_on = 'account_id', right_on = 'accountid')
zaw_acc = zaw_acc.merge(product_dwh[['product_id','product_level2']],on='product_id',how='left')
zaw_acc = zaw_acc.merge(fixdep[['accountid','status']],on='accountid',how='left').fillna('active')
zaw_acc['pdt_acc'] = zaw_acc['product_level2'] + '_' + zaw_acc['account_id'].str.slice(-4) + '_' + zaw_acc['status']

In [ ]:
zaw_acc.head()

In [ ]:
zaw_acc.loc[zaw_acc['product_level2']=='Fixed Deposit'].groupby('status').agg({'accountid':'nunique'})

In [ ]:
zaw_acc_bal = zaw_acc.groupby('pdt_acc').agg({'closing_balance_mmk':'mean'})/1e6
zaw_acc_bal.head()

In [ ]:
zaw_acc_above_zero = zaw_acc_bal.loc[zaw_acc_bal['closing_balance_mmk']>0]
zaw_acc_above_zero.head()

In [ ]:
zaw_acc_above_zero

In [ ]:
print(zaw_acc_bal.reset_index().pdt_acc.nunique())
print(zaw_acc_above_zero.reset_index().pdt_acc.nunique())

In [ ]:
zaw_acc.head()

In [ ]:
fbe_account.loc[fbe_account['cif']=='10183500'].head()

In [ ]:
fbe_account.columns

In [ ]:
fbe_customer.columns

In [ ]:
fbe_customer.partysubtype.unique()